# Eurybia - dataprep for US car accidents
This notebook describes the data preparation leading to the dataset in "US_Accidents_extract.csv", used in some of our tutorials.  


The original dataset was taken from the Kaggle [US car accidents dataset](https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents).

---
Acknowledgements
- Moosavi, Sobhan, Mohammad Hossein Samavatian, Srinivasan Parthasarathy, and Rajiv Ramnath. “A Countrywide Traffic Accident Dataset.”, 2019.
- Moosavi, Sobhan, Mohammad Hossein Samavatian, Srinivasan Parthasarathy, Radu Teodorescu, and Rajiv Ramnath. "Accident Risk Prediction based on Heterogeneous Sparse Data: New Dataset and Insights." In proceedings of the 27th ACM SIGSPATIAL International Conference on Advances in Geographic Information Systems, ACM, 2019.
---

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import category_encoders as ce

### Extract the zipped dataset if you haven't already done so

In [2]:
# from zipfile import ZipFile
# with ZipFile('/tmp/archive.zip', 'r') as zipObj:
#     zipObj.extractall()

### Load it up

In [ ]:
data = pd.read_csv('/tmp/US_Accidents_Dec20_updated.csv')

In [ ]:
print(data.shape)
print(data.columns)

In [5]:
feats_to_keep = ['Start_Lat','Start_Lng','Distance(mi)','Temperature(F)','Humidity(%)','Visibility(mi)',
 'Wind_Direction','Weather_Condition','day_of_week_acc','Nautical_Twilight','season_acc','cible','year_acc']

### Create a target column  
Here we regroup the severity modalities into two classes : benign to moderate severity (<= 2) on one side, serious and above on the other (>2)

In [6]:
print(data.Severity.value_counts())
cond = [data.Severity <= 2]
choice = ['0']
data['cible'] = np.select(cond, choice, default = '1')
data['cible'].value_counts(normalize = True)*100

2    1212382
3     161052
4     114452
1      28178
Name: Severity, dtype: int64


0    81.82768
1    18.17232
Name: cible, dtype: float64

### Rework the dates  
Here we build a "day of week", a "season" and a "year" feature. This will help us detect and analyze bias or trends that occur on those timescales.  
For example, we can then measure the drift between two same seasons of consecutive years to avoid seasonal bias.  
We could also aggregate by year and mesure the drift from year to year.

In [7]:
date = ['Start_Time', 'End_Time']
for d in date:
    data[d] = pd.to_datetime(data[d])

In [8]:
data['month_acc'] = data['Start_Time'].dt.month
data['day_of_week_acc'] = data['Start_Time'].dt.dayofweek
data['year_acc'] = data['Start_Time'].dt.year
cond = [data.month_acc.isin([12,1,2]), 
       data.month_acc.isin([3,4,5]), 
       data.month_acc.isin([6,7,8]), 
       data.month_acc.isin([9,10,11])]
choix = ['winter','spring','summer','autumn']
data['season_acc'] = np.select(cond, choix, default = 'NR')

### Managing missing values  
The following short analysis, using the "year" feature we just created, lets us see how the ratio of missing values evolved over time.  

In [10]:
missing_val = pd.DataFrame()
year = np.unique(data.year_acc)
for y in year:
    sub = data[data.year_acc == y]
    missing_val_y = pd.DataFrame(sub.isnull().sum().sort_values(ascending=False)/sub.shape[0]*100)
    missing_val_y.columns = ['taux_miss_'+str(y)]
    missing_val = pd.concat([missing_val, missing_val_y], axis = 1)

In [11]:
missing_val['filtre'] = missing_val.taux_miss_2016+missing_val.taux_miss_2017+missing_val.taux_miss_2018+missing_val.taux_miss_2019+missing_val.taux_miss_2020
missing_val[missing_val.filtre > 0][['taux_miss_2016','taux_miss_2017','taux_miss_2018','taux_miss_2019','taux_miss_2020']]

,taux_miss_2016,taux_miss_2017,taux_miss_2018,taux_miss_2019,taux_miss_2020
Precipitation(in),89.965591,87.001099,85.599871,21.002628,6.134666
Wind_Chill(F),88.079644,82.493136,77.976590,12.791666,3.989050
Number,78.399381,78.707106,78.010136,74.470150,61.636791
Wind_Speed(mph),17.903731,16.864885,18.997101,6.609569,3.554368
Visibility(mi),2.339841,2.651985,2.958619,2.746665,3.115117
Weather_Condition,2.316644,2.685495,3.050271,2.777226,3.056228
Humidity(%),2.173594,2.322765,2.691451,2.878459,3.391029
Temperature(F),2.049101,2.228702,2.578234,2.771496,3.177051
Pressure(in),1.597526,1.930640,2.472205,2.215669,2.664824
Wind_Direction,0.904697,1.142864,1.674893,3.026298,3.556906


$\require{color}$
$\colorbox{red}{The percentage of missing values, aggregated by year, is far from constant. This is a preliminary sign of data drift.}$

### Final dataset features

In [12]:
data = data[feats_to_keep]

### Quantitative features  

In [13]:
for v in ['Distance(mi)','Temperature(F)','Humidity(%)','Visibility(mi)']:
    data[v] = np.round(data[v],0)

In [14]:
data['Start_Lat'] = np.round(data['Start_Lat'],1)
data['Start_Lng'] = np.round(data['Start_Lng'],1)

### Categorical features  
Here we try to limit the number of modalities in each categorical feature.   
Reducing the cardinality of each feature down to a reasonable number (here we chose 10) helps with the training of many supervised models, namely CatBoost in the present case. So does encoding of categorical features (here, Wind_Direction).  
We also rework some labels for uniformity's sake. 

In [15]:
cond = [data['Weather_Condition'].isin(list(data.Weather_Condition.value_counts().index[0:9]))]
choix = [data['Weather_Condition']]
data['Weather_Condition'] = np.select(cond, choix, default = 'Other')

In [16]:
data['Wind_Direction'] = data['Wind_Direction'].fillna('NR')
data['Wind_Direction'] = data['Wind_Direction'].apply(lambda row: row.lower())

In [17]:
cond = [data.Wind_Direction == 'south',
       data.Wind_Direction == 'west',
       data.Wind_Direction == 'north',
        data.Wind_Direction == 'east',
        data.Wind_Direction == 'var',
       ]
choice = ['s','w','n','e','variable']
data['Wind_Direction'] = np.select(cond, choice, default = data['Wind_Direction'])

In [ ]:
data['Wind_Direction'] = ce.CatBoostEncoder(cols=['Wind_Direction'])\
.fit_transform(data[['Wind_Direction']], data.cible.astype('float'))

### Sampling  
For the purpose of our tutorials, a sample size of ~50000 is sufficient.  
The following few steps reduce the sample size down to about this number, and balance the number of samples per year, in an effort to reduce this source of bias before training a model or producing a quantitative analysis.

In [31]:
sampled_data = pd.DataFrame()
annee = np.unique(data.year_acc)
for a in annee:
    sub = data[data.year_acc == a]
    sub = sub.reset_index(drop = True)
    tir = np.random.choice(a = sub.shape[0], size = int(round(sub.shape[0]*0.2)), replace = False)
    sampled_data = pd.concat([sampled_data, sub.iloc[tir,:]], axis = 0)
    sampled_data = sampled_data.reset_index(drop = True)

In [33]:
sampled_data = sampled_data.iloc[np.random.choice(size = 50000, a = sampled_data.index, replace = False),:]
sampled_data = sampled_data.reset_index(drop  = True)

In [34]:
sampled_data = pd.concat([sampled_data.iloc[np.random.choice(sampled_data[sampled_data.year_acc == 2020].index, size = 8000, replace = False),:]\
           ,sampled_data[sampled_data.year_acc != 2020]], axis = 0)

### Let us have a final look at our data :

In [35]:
sampled_data.head(3)

,Start_Lat,Start_Lng,Distance(mi),Temperature(F),Humidity(%),Visibility(mi),Wind_Direction,Weather_Condition,day_of_week_acc,Nautical_Twilight,season_acc,cible,year_acc
37331,47.0,-114.3,0.0,15.0,95.0,10.0,0.132205,Fair,0,Night,winter,0,2020
32661,33.9,-117.9,0.0,67.0,51.0,10.0,0.140612,Fair,3,Day,winter,0,2020
7027,40.2,-79.6,1.0,41.0,42.0,10.0,0.114082,Partly Cloudy,0,Day,autumn,0,2020


In [36]:
sampled_data.describe()

,Start_Lat,Start_Lng,Distance(mi),Temperature(F),Humidity(%),Visibility(mi),Wind_Direction,day_of_week_acc,year_acc
count,32003.000000,32003.000000,32003.000000,31134.000000,31092.000000,31078.000000,32003.000000,32003.000000,32003.000000
mean,37.246477,-99.243827,0.566509,60.403707,64.004213,9.169187,0.186780,2.541762,2018.332000
std,5.079399,18.433532,1.693327,18.694226,23.350920,3.198350,0.064868,1.795885,1.363174
min,24.700000,-124.500000,0.000000,-25.000000,1.000000,0.000000,0.060574,0.000000,2016.000000
25%,33.900000,-118.200000,0.000000,48.000000,47.000000,10.000000,0.137373,1.000000,2017.000000
50%,37.700000,-95.400000,0.000000,62.000000,67.000000,10.000000,0.160983,2.000000,2019.000000
75%,40.900000,-81.400000,1.000000,74.000000,83.000000,10.000000,0.232959,4.000000,2019.000000
max,49.000000,-68.400000,112.000000,112.000000,100.000000,120.000000,0.590862,6.000000,2020.000000


### Write the sample to disk

In [ ]:
# sampled_data.to_csv('US_Accidents_extract.csv', index = False)